In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb import setup_1,setup_2,setup_2_old,setup_1_old
import warnings
warnings.filterwarnings("ignore")

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

#ativo = 'EURUSD'
ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 70000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2022-07-12 16:07:00,99265.0,99270.0,99195.0,99200.0,24888
1,2022-07-12 16:08:00,99205.0,99235.0,99175.0,99220.0,29412
2,2022-07-12 16:09:00,99220.0,99220.0,99185.0,99205.0,15143
3,2022-07-12 16:10:00,99205.0,99245.0,99180.0,99230.0,25505
4,2022-07-12 16:11:00,99230.0,99235.0,99180.0,99235.0,17137
...,...,...,...,...,...,...
69995,2023-01-13 18:21:00,112055.0,112060.0,111985.0,112010.0,3772
69996,2023-01-13 18:22:00,112010.0,112080.0,111970.0,111980.0,5145
69997,2023-01-13 18:23:00,111985.0,112025.0,111960.0,112025.0,2927
69998,2023-01-13 18:24:00,112010.0,112030.0,111955.0,111995.0,2319


In [6]:
time_period = 40
var_bb = 2.3
pontos = 150
rate_stop = 1
rate_tp = 2
timeframe = 5

for time_period in [14,20,30,50]:
    for var_bb in [1.8,2,2.2]:
        for pontos in [60,100,150,200,250,300]: 
            df_acao = setup_2(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_stop = rate_stop,rate_tp = rate_tp,timeframe = 5)
            #df_acao = setup_1_old(df,var_bb = 2,time_period = time_period,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)
            #df_acao = df[(df['acao'] == 'call')]

            df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
            df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])


            print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos} -- Timeframe: {timeframe}')
            df_new_hora = df_acao[['Hora_h','resultado_binario']]
            df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

            #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
            print(df_new_hora)

            '''df_acao = setup_2(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_stop = rate_stop,rate_tp = rate_tp,timeframe = 5)
            #df_acao = df[(df['acao'] == 'call')]

            df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
            df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])


            print('Agrupado por Hora')
            df_new_hora = df_acao[['Hora_h','resultado_binario']]
            df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

            #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
            print(df_new_hora)'''


Time_period: 14 -- var_bb: 1.8 -- pontos: 60 -- Timeframe: 5
  Hora_h  qtd_operacoes  taxa_acerto
0     09             73     0.342466
1     10             58     0.327586
2     11             54     0.277778
3     12             91     0.406593
4     13             99     0.323232
5     14            106     0.198113
6     15            102     0.333333
7     16            105     0.371429
8     17            138     0.333333
9     18             17     0.470588
Time_period: 14 -- var_bb: 1.8 -- pontos: 100 -- Timeframe: 5
  Hora_h  qtd_operacoes  taxa_acerto
0     09             73     0.328767
1     10             58     0.310345
2     11             54     0.351852
3     12             91     0.439560
4     13             99     0.282828
5     14            106     0.273585
6     15            102     0.284314
7     16            105     0.457143
8     17            138     0.369565
9     18             17     0.294118
Time_period: 14 -- var_bb: 1.8 -- pontos: 150 -- Timeframe: 5
 

In [9]:
df_acao[['Data','pos_BB_inicial','acao','resultado_binario']].tail(10)

,Data,pos_BB_inicial,acao,resultado_binario
39868,2023-01-11 17:20:00,0,sell,0.0
39874,2023-01-11 17:50:00,0,sell,0.0
39909,2023-01-12 11:20:00,0,call,1.0
39916,2023-01-12 11:55:00,0,call,0.0
39919,2023-01-12 12:10:00,0,call,1.0
39932,2023-01-12 13:15:00,0,sell,0.0
39959,2023-01-12 15:30:00,0,call,0.0
39973,2023-01-12 16:40:00,0,call,0.0
39980,2023-01-12 17:15:00,0,call,1.0
39996,2023-01-13 09:10:00,0,call,0.0
